### Exercise x.x
Implement a Class LLSQ that can do the calculation of the linear least squares algorithm with given Data
The class should be initialized with a list of functions that constitute the function to fit. e.g. y=a * np.sin + b * np.cos -> [np.sin, np.cos]
One class method should involve the calculation of the covariance matrix and calculation of the fit parameters.

In [1]:
import numpy as np

In [2]:
### your code here
class linleastsquares():
   
    def __init__(self, functionlist):
        
        '''
        Initiate the object with the wanted list of functions
        '''
        self.functionlist=functionlist
        
    
    def fit(self, x_values, y_values, y_errors, print_matrix=False):
        
        """
        Calculate the parameters for the linear leastsquares model
        of functions provided as argumen functionlist for the x_values
        and y_values with errors y_errors
        """        
        # set dimension of the designmatrix
        dim = (len(x_values), len(self.functionlist))
        A = np.zeros(dim)

        # fill design matrix with function values
        for i, func in enumerate(self.functionlist):
            A[:,i] = func(x_values)
        #cast to np.matrix type
        A = np.matrix(A)

        if print_matrix is True:
            print(A)
            
        # cast y-values into a column vector
        y = np.matrix(y_values).T
        
        # create diagonal weighting matrix
        W = np.matrix(np.diag(1/(y_errors**2)))

        # do the calculations weighted LinSquares
        invATA = (A.T * W * A).I
        params = invATA * A.T * W * y
        cov = invATA

        return np.array(params.flat), cov

In [4]:
# read data
# maybe get different data -> different fit functions
x, y  = np.genfromtxt("data/llsq/data.txt", unpack=True)

y_err = np.ones(len(y)) * 0.011

# linear leastsquares fit object for a*sin + b*cos
llsq = linleastsquares([np.sin, np.cos])

(a, b), cov_ab = llsq.fit(x, y, y_err, True)


[[ 0.          1.        ]
 [-0.98803162  0.15425145]
 [-0.30481062 -0.95241298]
 [ 0.89399666 -0.44807362]
 [ 0.58061118  0.81418097]
 [-0.71487643  0.69925081]
 [-0.80115264 -0.59846007]
 [ 0.46771852 -0.88387747]
 [ 0.94544515  0.32578131]
 [-0.17604595  0.98438195]
 [-0.99975584 -0.02209662]
 [-0.13238163 -0.99119882]]


### Error propagation
Statistical errors from measurements/fits have to be taken into account when calculating values from fit parameters

In [11]:
# Berechnen von A0 und delta:
A0 = np.sqrt(a**2 + b**2)
delta = -np.arctan2(a,b)

# Error propagation with BVB.T B is Jacobimatrix of the functions
B = np.matrix([[a/np.sqrt(a**2+b**2), b/np.sqrt(a**2+b**2)],
               [b/(b**2 + a**2), -a/(a**2 + b**2)]])

cov_2 = B*cov_ab*B.T
print('Parameters')
print("a =", a, u"±", np.sqrt(cov_ab[0,0]))
print("b =", b, u"±", np.sqrt(cov_ab[1,1]))
print('___________________')
print('Pearson correlation of fit values')
print(u"ρ =", cov_ab[0,1]/(np.sqrt(cov_ab[0,0] * cov_ab[1,1])))
print('___________________')
print('Parameter covariance matrix')
print("Cov(a,b) =\n", cov_ab)
print('___________________')
print('Calculated values with errors')
print("A0 =", A0, u"±", np.sqrt(cov_2[0,0]))
print(u"δ =", delta, u"±", np.sqrt(cov_2[1,1]))
print('___________________')
print('Pearson correlation of calculated values')
print(u"ρ =", cov_2[0,1]/(np.sqrt(cov_2[0,0] * cov_2[1,1])))
print('___________________')
print('Calculated values covariance matrix')
print(u"Cov(A0, δ) =\n", cov_2)

Parameters
a = 0.013282587762236876 ± 0.004682602990324402
b = -0.0073876518589510165 ± 0.004321125737056743
___________________
Pearson correlation of fit values
ρ = -0.010741281100605746
___________________
Parameter covariance matrix
Cov(a,b) =
 [[ 2.19267708e-05 -2.17340331e-07]
 [-2.17340331e-07  1.86721276e-05]]
___________________
Calculated values with errors
A0 = 0.015198833430582336 ± 0.004619791184445168
δ = -2.078380050779103 ± 0.28872049093935437
___________________
Pearson correlation of calculated values
ρ = -0.0625414297454016
___________________
Calculated values covariance matrix
Cov(A0, δ) =
 [[ 2.13424706e-05 -8.34195338e-05]
 [-8.34195338e-05  8.33595219e-02]]
